<a href="https://colab.research.google.com/github/Hemanthnaidu3737/Sentiment-Analysis-using-BERT-and-Transformer/blob/main/Sentiment_Analysis_using_BERT_and_Transformers_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers tensorflow datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00


In [ ]:
# Basics
import os
import json
import numpy as np
import pandas as pd
import torch

# Hugging Face Transformers
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import AdamW

# PyTorch Libraries
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Scikit-learn for Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# NLP Preprocessing
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Enable GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("/content/Train.csv")


Saving twitter_training.csv to twitter_training (1).csv


In [ ]:
# Load datasets
train_path = "/content/Train.csv"
from datasets import load_dataset

ds = load_dataset("ashraq/hotel-reviews")
train_path = "/content/twitter_training.csv"
valid_path = "/content/twitter_validation.csv"

df_train = pd.read_csv(train_path, encoding='latin1')
df_valid = pd.read_csv(valid_path, encoding='latin1')

# Display dataset samples
print("Training Data Sample:")
print(df_train.head())

print("Validation Data Sample:")
print(df_valid.head())


README.md:   0%|          | 0.00/548 [00:00<?, ?B/s]

(…)-00000-of-00001-d30d4a38ef4cc157.parquet:   0%|          | 0.00/6.10M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93757 [00:00<?, ? examples/s]

Training Data Sample:
   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

  im getting on borderlands and i will murder you all ,  
0  I am coming to the borders and I will kill you...     
1  im getting on borderlands and i will kill you ...     
2  im coming on borderlands and i will murder you...     
3  im getting on borderlands 2 and i will murder ...     
4  im getting into borderlands and i can murder y...     
Validation Data Sample:
   3364   Facebook Irrelevant  \
0   352     Amazon    Neutral   
1  8312  Microsoft   Negative   
2  4371      CS-GO   Negative   
3  4433     Google    Neutral   
4  6273       FIFA   Negative   

  I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tomâs great auntie as âHayley canât get out of bedâ and told to his gra

In [ ]:
# Rename columns appropriately
df_train.columns = ["ID", "Topic", "Sentiment", "Text"]
df_valid.columns = ["ID", "Topic", "Sentiment", "Text"]

# Drop unnecessary ID column
df_train = df_train[["Topic", "Sentiment", "Text"]]
df_valid = df_valid[["Topic", "Sentiment", "Text"]]

print(df_train.head())


         Topic Sentiment                                               Text
0  Borderlands  Positive  I am coming to the borders and I will kill you...
1  Borderlands  Positive  im getting on borderlands and i will kill you ...
2  Borderlands  Positive  im coming on borderlands and i will murder you...
3  Borderlands  Positive  im getting on borderlands 2 and i will murder ...
4  Borderlands  Positive  im getting into borderlands and i can murder y...


In [ ]:
label_mapping = {"Positive": 1, "Negative": 0, "Neutral": 2, "Irrelevant": 3}
df_train["Sentiment"] = df_train["Sentiment"].map(label_mapping)
df_valid["Sentiment"] = df_valid["Sentiment"].map(label_mapping)

print(df_train.head())


         Topic  Sentiment                                               Text
0  Borderlands          1  I am coming to the borders and I will kill you...
1  Borderlands          1  im getting on borderlands and i will kill you ...
2  Borderlands          1  im coming on borderlands and i will murder you...
3  Borderlands          1  im getting on borderlands 2 and i will murder ...
4  Borderlands          1  im getting into borderlands and i can murder y...


In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize text
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Rename columns appropriately
df_train.columns = ["ID", "Topic", "Sentiment", "Text"]
df_valid.columns = ["ID", "Topic", "Sentiment", "Text"]

# Drop unnecessary ID column but keep 'Text' column
df_train = df_train[["Topic", "Sentiment", "Text"]] # Keep 'Text' column
df_valid = df_valid[["Topic", "Sentiment", "Text"]] # Keep 'Text' column

print(df_train.head())

print("Tokenization Complete!")

         Topic Sentiment                                               Text
0  Borderlands  Positive  I am coming to the borders and I will kill you...
1  Borderlands  Positive  im getting on borderlands and i will kill you ...
2  Borderlands  Positive  im coming on borderlands and i will murder you...
3  Borderlands  Positive  im getting on borderlands 2 and i will murder ...
4  Borderlands  Positive  im getting into borderlands and i can murder y...
Tokenization Complete!


In [ ]:
label_mapping = {"Positive": 1, "Negative": 0, "Neutral": 2, "Irrelevant": 3}
df_train["Sentiment"] = df_train["Sentiment"].map(label_mapping)
df_valid["Sentiment"] = df_valid["Sentiment"].map(label_mapping)

print("Sentiment Labels Mapped:", df_train["Sentiment"].unique())


Sentiment Labels Mapped: [1 2 0 3]


<ipython-input-40-0f3c8e105ff0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["Sentiment"] = df_train["Sentiment"].map(label_mapping)


In [ ]:
# Tokenize the text column
# Convert all elements in the 'Text' column to strings before tokenization
train_encodings = tokenizer(df_train["Text"].astype(str).tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt")
valid_encodings = tokenizer(df_valid["Text"].astype(str).tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt")

print("Tokenization Complete!")


Tokenization Complete!


In [ ]:
import torch
from torch.utils.data import Dataset

# Custom dataset class for PyTorch
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Create dataset objects for training and validation
train_dataset = SentimentDataset(train_encodings, df_train["Sentiment"].tolist())
valid_dataset = SentimentDataset(valid_encodings, df_valid["Sentiment"].tolist())

print("Datasets Ready for Training!")


Datasets Ready for Training!


In [ ]:
from transformers import BertForSequenceClassification

# Load BERT with the number of sentiment labels
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)  # 4 classes (Positive, Negative, Neutral, Irrelevant)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("BERT Model Loaded!")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Model Loaded!


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",          # Save model here
    num_train_epochs=3,              # Train for 3 epochs
    per_device_train_batch_size=8,   # Batch size during training
    per_device_eval_batch_size=8,    # Batch size during evaluation
    warmup_steps=500,                # Warmup steps for scheduler
    weight_decay=0.01,               # Regularization to prevent overfitting
    logging_dir="./logs",            # Log directory
    logging_steps=10,                 # Log progress every 10 steps
    evaluation_strategy="epoch",      # Evaluate after every epoch
)

print("Training Arguments Set!")


Training Arguments Set!


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="google-bert/bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pretrained sentiment model
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

print("Pretrained Sentiment Model Loaded!")


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Pretrained Sentiment Model Loaded!


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax().item()

    sentiment_labels = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    return sentiment_labels[predicted_class]

# Example Sentiment Prediction
text = "I love this product! It's amazing."
print("Sentiment:", predict_sentiment(text))


Sentiment: Very Positive


In [ ]:
import random

# Define intents and responses
intent_responses = {
    "greeting": ["Hello! How can I help you?", "Hey there! 😊", "Hi! What can I do for you?"],
    "goodbye": ["Goodbye! Have a great day! 😊", "See you later!", "Bye! Take care!"],
    "movie_info": ["RRR is a popular movie directed by S.S. Rajamouli.",
                   "RRR features NTR and Ram Charan in lead roles.",
                   "Would you like to know more about RRR?"],
    "movie_dialogue": ["Here's a famous dialogue from RRR: 'Yuddham Yuddham Yuddham!'",
                       "RRR dialogue: 'Rise, Roar, Revolt!'"],
    "joke": ["Why don’t scientists trust atoms? Because they make up everything!",
             "I told my wife she was drawing her eyebrows too high. She looked surprised! 😂"],
    "help": ["Sure! How can I assist you?", "Let me know what you need help with!"]
}


In [ ]:
def detect_intent(user_input):
    user_input = user_input.lower()

    if any(word in user_input for word in ["hi", "hello", "hey"]):
        return "greeting"
    if any(word in user_input for word in ["book", "novel", "reading"]):
        return "books"
    if any(word in user_input for word in ["movie", "film", "cinema", "watch"]):
        return "movies"
    if any(word in user_input for word in ["music", "song", "singer", "band"]):
        return "music"
    if any(word in user_input for word in ["sport", "football", "cricket", "game"]):
        return "sports"
    if any(word in user_input for word in ["travel", "trip", "destination"]):
        return "travel"
    if any(word in user_input for word in ["breakup", "relationship", "heartbroken"]):
        return "relationship"
    if any(word in user_input for word in ["joke", "funny", "laugh"]):
        return "joke"
    if any(word in user_input for word in ["help", "assist", "support"]):
        return "help"

    return None  # No specific intent detected


In [ ]:
def chatbot_response(user_input):
    # Check if the input matches any intent
    intent = detect_intent(user_input)

    if intent:
        return random.choice(intent_responses[intent])  # Return a random response for detected intent

    # If no intent detected, use sentiment analysis
    sentiment = predict_sentiment(user_input)

    sentiment_responses = {
        "Very Negative": "I'm sorry you feel this way. How can I help?",
        "Negative": "I understand. Let me know how I can assist you.",
        "Neutral": "Thanks for your message. Do you have any questions?",
        "Positive": "I'm glad to hear that! 😊",
        "Very Positive": "That's awesome! 🎉 How else can I help?"
    }

    return sentiment_responses.get(sentiment, "I'm not sure how to respond to that. Can you rephrase?")


In [ ]:
import random

# Intent responses with multiple unique answers
intent_responses = {
    "greeting": [
        "Hello! How can I help you?",
        "Hey there! 😊",
        "Hi! What can I do for you?",
        "Nice to meet you! What’s on your mind?"
    ],
    "books": [
        "I love books too! 📚 What's your favorite genre?",
        "Reading is amazing! Do you prefer fiction or non-fiction?",
        "Books open up new worlds! Which book have you read recently?",
        "There are so many great books! Do you like classics or modern novels?",
        "A book a day keeps boredom away! What are you currently reading?"
    ],
    "movies": [
        "Movies are great! 🎬 Do you have a favorite director?",
        "I love movies too! Do you prefer action, drama, or sci-fi?",
        "What was the last movie you watched?"
    ],
    "music": [
        "Music makes life better! 🎶 What’s your favorite song?",
        "I love music too! Do you listen to classical or modern songs?",
        "Which genre do you like the most—rock, pop, jazz, or classical?"
    ],
    "sports": [
        "Sports are fun! ⚽ Do you play any sports?",
        "Are you a fan of cricket, football, or basketball?",
        "Which team do you support?"
    ],
    "travel": [
        "Traveling is amazing! ✈️ What's your dream destination?",
        "Do you prefer beaches or mountains?",
        "If you could travel anywhere today, where would you go?"
    ],
    "relationship": [
        "Breakups are tough. Stay strong. ❤️ Want to talk about it?",
        "I'm here for you. Things will get better. 💖",
        "Would you like some advice or just someone to listen?"
    ],
    "joke": [
        "Why don’t scientists trust atoms? Because they make up everything!",
        "I told my wife she was drawing her eyebrows too high. She looked surprised! 😂",
        "Why did the scarecrow win an award? Because he was outstanding in his field!"
    ],
    "help": [
        "Sure! How can I assist you?",
        "Let me know what you need help with!",
        "I'm here to help! Ask me anything."
    ]
}


In [ ]:
def detect_intent(user_input):
    user_input = user_input.lower()

    if any(word in user_input for word in ["hi", "hello", "hey"]):
        return "greeting"
    if any(word in user_input for word in ["bye", "goodbye", "see you"]):
        return "goodbye"
    if any(word in user_input for word in ["tell me about rrr", "what is rrr", "rrr movie"]):
        return "movie_info"
    if any(word in user_input for word in ["dialogue", "quote", "famous line"]):
        return "movie_dialogue"
    if any(word in user_input for word in ["joke", "funny", "laugh"]):
        return "joke"
    if any(word in user_input for word in ["help", "assist", "support"]):
        return "help"
    if any(word in user_input for word in ["book", "novel", "reading"]):
        return "books"
    if any(word in user_input for word in ["music", "song", "singer", "band"]):
        return "music"
    if any(word in user_input for word in ["sport", "football", "cricket", "game"]):
        return "sports"
    if any(word in user_input for word in ["travel", "trip", "destination"]):
        return "travel"
    if any(word in user_input for word in ["breakup", "relationship", "heartbroken"]):
        return "relationship"

    return None  # No specific intent detected


In [ ]:
def chatbot_response(user_input):
    # Check if the input matches any intent
    intent = detect_intent(user_input)

    if intent:
        return random.choice(intent_responses[intent])

    # If no intent detected, use sentiment analysis
    sentiment = predict_sentiment(user_input)

    sentiment_responses = {
        "Very Negative": "I'm sorry you feel this way. Do you want to talk about it?",
        "Negative": "I understand. Let me know how I can assist you.",
        "Neutral": "Thanks for your message. Do you have any questions?",
        "Positive": "I'm glad to hear that! 😊",
        "Very Positive": "That's awesome! 🎉 How else can I help?"
    }

    return sentiment_responses.get(sentiment, "I'm not sure how to respond to that. Can you rephrase?")


In [ ]:
# Store user memory
user_memory = {"interests": [], "last_topic": None, "last_response": {}}

def update_memory(user_id, topic):
    if topic not in user_memory["interests"]:
        user_memory["interests"].append(topic)
    user_memory["last_topic"] = topic

def get_unique_response(intent):
    responses = intent_responses[intent]

    # Get the last response used
    last_response = user_memory["last_response"].get(intent, None)

    # Filter out the last response to avoid immediate repetition
    available_responses = [resp for resp in responses if resp != last_response]

    # If all responses are used, reset the cycle
    if not available_responses:
        available_responses = responses

    # Pick a new response and update memory
    new_response = random.choice(available_responses)
    user_memory["last_response"][intent] = new_response

    return new_response

def chatbot_response_with_memory(user_id, user_input):
    global user_memory

    # Detect intent
    intent = detect_intent(user_input)

    if intent:
        update_memory(user_id, intent)

        # Get a unique response
        return get_unique_response(intent)

    # If user repeats a topic with no new intent, ask them for details
    if user_memory["last_topic"]:
        return f"You mentioned {user_memory['last_topic']} earlier. What specifically do you want to know about it?"

    # Use sentiment analysis if no intent is detected
    sentiment = predict_sentiment(user_input)

    sentiment_responses = {
        "Very Negative": "I'm sorry you feel this way. Do you want to talk about it?",
        "Negative": "I understand. Let me know how I can assist you.",
        "Neutral": "Thanks for your message. Do you have any questions?",
        "Positive": "I'm glad to hear that! 😊",
        "Very Positive": "That's awesome! 🎉 How else can I help?"
    }

    return sentiment_responses.get(sentiment, "I'm not sure how to respond to that. Can you rephrase?")


In [ ]:
print("🤖 Chatbot is ready! Type 'exit' or 'bye' to stop.")

while True:
    user_message = input("You: ")
    if user_message.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye! Have a great day! 😊")
        break
    print("Chatbot:", chatbot_response_with_memory("user1", user_message))


🤖 Chatbot is ready! Type 'exit' or 'bye' to stop.
Chatbot: Hello! How can I help you?
Chatbot: A book a day keeps boredom away! What are you currently reading?
Chatbot: There are so many great books! Do you like classics or modern novels?
Chatbot: You mentioned books earlier. What specifically do you want to know about it?
Chatbot: Sports are fun! ⚽ Do you play any sports?
Chatbot: Hey there! 😊
Chatbot: You mentioned greeting earlier. What specifically do you want to know about it?
Chatbot: Sure! How can I assist you?
